In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init

In [3]:
# df = data.getInitData()

In [4]:
# data.mk_processedData(df)

In [5]:
# first_np, second_np, label_np = utils.process(df)

In [6]:
first_np, second_np, label_np  = data.getData()

In [7]:
first_np, second_np = utils.process_splitted(first_np, second_np)

In [8]:
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)

In [9]:
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

In [10]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 132 pretrained vectors found.


In [11]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

In [12]:
train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [13]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [14]:
# PyTorch TensorBoard support
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# writer = SummaryWriter('ocr/tests')

In [15]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [ ]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=len(ch2idx),
    embed_dim=200,
    hidden_size=100,
    num_classes=2,
    rnn_layers=3,
    dropout=0.2,
    learning_rate=0.001,
    model_name="RNN",
    optim_name="Adam",
    loss_fn_name="CEL",
    pretrained_model=pretrained_word2vec,
    freeze_embedding=False,
    device=device,
)

print(model)

In [ ]:
trainer.train()

In [16]:
import torch.nn as nn
import ocr_model as om
import torch.optim as optim

In [17]:
model = om.OCR(
            len(ch2idx),                                    # input size = vocab size
            200,                                            # vector dimension size
            100,                                            # hidden size
            2,                                              # number of class
            2,                                              # lstm layer
            0.0,                                            # drouput value
            pretrained_embedding=pretrained_word2vec,       # pretrained model (optional)
            freeze_embedding=False,                         # false; updates tensor
        )

doing with pretrained model!!!


In [18]:
model.to(device)

OCR(
  (emb): Embedding(132, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, num_layers=2, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, num_layers=2, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [19]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [20]:
loss_fn = nn.CrossEntropyLoss()

In [26]:
for step, batch in enumerate(train_dataloader):
    firsts, seconds, labels = tuple(t.to(device) for t in batch)

    model.zero_grad()

    logits = model(firsts, seconds)

    loss = loss_fn(logits, labels)

    preds = torch.argmax(logits, dim=1).flatten()

    accuracy = (preds == labels).cpu().numpy().mean() * 100

    print(loss.item())
    print()
    print(accuracy)

    # Perform a backward pass to calculate gradients
    loss.backward()

    # Update parameters
    optimizer.step()

    break

0.6963645219802856

48.0
